In [1]:
#Connect to GitHub data (STILL LOADS FLOWER DATA)

!wget --no-check-certificate \
    https://github.com/GGEDPFJC/WaterColumn_Classifier_V1/master/train.zip?raw=true \
    -O /tmp/train.zip

--2021-11-03 02:43:26--  https://github.com/GGEDPFJC/WaterColumn_Classifier_V1/master/train.zip?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-11-03 02:43:26 ERROR 404: Not Found.



In [ ]:
#Load training and validation data

import os
import zipfile

local_zip = '/tmp/train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
local_zip = '/tmp/valid.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
# Directory with our training plume pictures
train_plume_dir = os.path.join('/tmp/train/plume')

# Directory with our training no plume pictures
train_noplume_dir = os.path.join('/tmp/train/noplume')

# Directory with our validation plume pictures
valid_plume_dir = os.path.join('/tmp/valid/plume')

# Directory with our validation no plume pictures
valid_noplume_dir = os.path.join('/tmp/valid/noplume')

In [ ]:
#Check filenames

train_plume_names = os.listdir(train_plume_dir)
print(train_plume_names[:10])

train_noplume_names = os.listdir(train_noplume_dir)
print(train_grass_names[:10])

validation_plume_names = os.listdir(valid_plume_dir)
print(validation_plume_hames[:10])

validation_noplume_names = os.listdir(valid_noplume_dir)
print(validation_noplume_names[:10])

In [ ]:
#Check number of files

print('total training plume images:', len(os.listdir(train_plume_dir)))
print('total training no plume images:', len(os.listdir(train_noplume_dir)))
print('total validation plume images:', len(os.listdir(valid_plume_dir)))
print('total validation no plume images:', len(os.listdir(valid_noplume_dir)))

In [ ]:
#Inspect images

%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# graph parameters
nrows = 4
ncols = 4

# iteration index
pic_index = 0

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_plume_pic = [os.path.join(train_plume_dir, fname) 
                for fname in train_plume_names[pic_index-8:pic_index]]
next_noplume_pic = [os.path.join(train_noplume_dir, fname) 
                for fname in train_noplume_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_plume_pic + next_noplume_pic):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
#Preprocess Images

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 120 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/train/',  # This is the source directory for training images
        classes = ['plume', 'no plume'],
        target_size=(200, 200),  # All images will be resized to 200x200
        batch_size=120,
        # Use binary labels
        class_mode='binary')

# Flow validation images in batches of 19 using valid_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/valid/',  # This is the source directory for training images
        classes = ['plume', 'no plume'],
        target_size=(200, 200),  # All images will be resized to 200x200
        batch_size=19,
        # Use binary labels
        class_mode='binary',
        shuffle=False)

In [ ]:
#Define Model

import tensorflow as tf
import numpy as np
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = (200,200,3)), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Train model (15 epochs in this line)

history = model.fit(train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

In [ ]:
#Evaluate accuracy

model.evaluate(validation_generator)

In [ ]:
#Use model for predictions

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(200, 200))
  x = image.img_to_array(img)
  plt.imshow(x/255.)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]<0.5:
    print(fn + " contains a bubble plume")
  else:
    print(fn + " does not contain a bubble plume")